# Heat Maps

In this noteboook , we produce a Heat map showing the influential regions in mathematics as time progresses.

In [7]:
import json
import pandas as pd
from folium import plugins
import folium
from datetime import datetime

In [21]:
data = pd.read_csv("data/data_descendants.csv")

In [22]:
data.head()

,id,name,thesis,school,country,year,subject,advisors,students,descendants
0,11,Alfred E. Engelbrecht,Coupled Bending and Torsional Free Vibration o...,Iowa State University,UnitedStates,1949.0,NaN,[1],[],0
1,4,Charles Joseph Thorne,The Approximate Solution of Linear Differentia...,Iowa State University,UnitedStates,1941.0,NaN,[239],"[19930, 19939]",2
2,1,Ernest Willard Anderson,Statics of Special Types of Homogeneous Elasti...,Iowa State University,UnitedStates,1933.0,74—Mechanics of deformable solids,[258],"[11, 28, 31]",3
3,7,Carl Eric Langenhop,Properties of Kernels of Integral Equations Wh...,Iowa State University,UnitedStates,1948.0,NaN,[281],"[48, 104689, 38, 33, 3026, 39, 32, 35]",37
4,6,William B. Stiles,Solutions of Clamped Plated Problems by Means ...,Iowa State University,UnitedStates,1945.0,NaN,[258],[],0


In [23]:
coords = pd.read_csv("data/coordinates.csv")

In [24]:
coords.head()

,school,lat,lon
0,Iowa State University,42.027961,-93.644737
1,University of Tennessee - Knoxville,35.951635,-83.930882
2,University of Illinois at Urbana-Champaign,40.099118,-88.238516
3,"University of California, Berkeley",37.875500,-122.239069
4,University of Wisconsin-Madison,43.080274,-89.430959


In [25]:
coords.shape

(3907, 3)

In [26]:
coords.dropna(inplace=True)

# Merging the data with coordinates

In [27]:
full_data = pd.merge(data,coords,on='school',how='left')
full_data.shape


(256769, 12)

In [28]:
full_data.shape

(256769, 12)

In [29]:
full_data.head()

,id,name,thesis,school,country,year,subject,advisors,students,descendants,lat,lon
0,11,Alfred E. Engelbrecht,Coupled Bending and Torsional Free Vibration o...,Iowa State University,UnitedStates,1949.0,NaN,[1],[],0,42.027961,-93.644737
1,4,Charles Joseph Thorne,The Approximate Solution of Linear Differentia...,Iowa State University,UnitedStates,1941.0,NaN,[239],"[19930, 19939]",2,42.027961,-93.644737
2,1,Ernest Willard Anderson,Statics of Special Types of Homogeneous Elasti...,Iowa State University,UnitedStates,1933.0,74—Mechanics of deformable solids,[258],"[11, 28, 31]",3,42.027961,-93.644737
3,7,Carl Eric Langenhop,Properties of Kernels of Integral Equations Wh...,Iowa State University,UnitedStates,1948.0,NaN,[281],"[48, 104689, 38, 33, 3026, 39, 32, 35]",37,42.027961,-93.644737
4,6,William B. Stiles,Solutions of Clamped Plated Problems by Means ...,Iowa State University,UnitedStates,1945.0,NaN,[258],[],0,42.027961,-93.644737


## Plotting

In [35]:
def plot_inf_univ(full_data, t0 = 1950, t1 = 1960, ignore_zeroes = True, show_map = True):
    period = full_data[ (full_data["year"]>=t0) & (full_data['year']<t1)]

    p = period[["school","lat","lon","descendants"]].groupby(['school'])
    p = p["descendants"].agg(['sum']).reset_index()
    p = pd.merge(p,coords,on='school',how='left')

    X = p[["lat","lon","sum"]]
    X.dropna(inplace=True)
    if ignore_zeroes:
        X = X[X["sum"] != 0]
        map = folium.Map(location = [5,30], tiles = "Cartodb dark_matter", zoom_start=2)

        heat_data = X.values.tolist()

        plugins.HeatMap(heat_data).add_to(map)

        map.save(f"heat_map_ignore_{t0}_to_{t1}.html")
        if show_map:
            display(map)
        return map
        
    else:
        map = folium.Map(location = [5,30], tiles = "Cartodb dark_matter", zoom_start=2)

        heat_data = X.values.tolist()

        plugins.HeatMap(heat_data).add_to(map)

        map.save(f"heat_map_{t0}_to_{t1}.html")
        if show_map:
            display(map)
        return map


In [39]:
map = plot_inf_univ(full_data, ignore_zeroes= True)


C:\Users\venka\AppData\Local\Temp\ipykernel_15780\2679485811.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)


# Heatmap with time experiments

In [40]:
def plot_inf_univ_withtime(full_data, t0 = 1950, t1 = 1960, ignore_zeroes = False, show_map = True):
    L=[]
    for t in range(t0,t1,5):
        period = full_data[ (full_data["year"]>=t) & (full_data['year']<t+5)]
        p = period[["school","lat","lon","descendants"]].groupby(['school'])
        p = p["descendants"].agg(['sum']).reset_index()
        p = pd.merge(p,coords,on='school',how='left')

        X = p[["lat","lon","sum"]]
        X.dropna(inplace=True)
        if ignore_zeroes:
            X = X[X["sum"] != 0]
        L.append(X.values.tolist())

    map = folium.Map(location = [5,30], tiles = "Cartodb dark_matter", zoom_start=2)


    plugins.HeatMapWithTime(L, auto_play=True, max_opacity=0.3).add_to(map)
    map.save(f"heat_time_{t0}_{t1}.html")
    if show_map:
        display(map)
    return map

In [42]:
plot_inf_univ_withtime(full_data, t0 = 1950, t1 = 2020, ignore_zeroes = False, show_map = True)


C:\Users\venka\AppData\Local\Temp\ipykernel_15780\3169883036.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)
C:\Users\venka\AppData\Local\Temp\ipykernel_15780\3169883036.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)
C:\Users\venka\AppData\Local\Temp\ipykernel_15780\3169883036.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)
C:\Users\venka\AppData\Local